# Springboard Data Science Career Track

## Capstone Project #2: Learning Neural Networks through Text Generation

## Extended Model #8: Changing Depth of + Memory Unites Within Word-Level Model

##### By Logan Larson

In [1]:
import numpy
from numpy import array
import string
import re
from random import randint
from pickle import load, dump
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Fu

In [2]:
# load text and convert to lowercase
filename = "AdventuresOfSherlockHolmes.txt"
raw_text = open(filename).read()
print(raw_text[:102])

To Sherlock Holmes she is always THE woman. I have seldom heard
him mention her under any other name. 


In [3]:
# convert to lowercase
raw_text = raw_text.lower()
# replace dashes
raw_text = raw_text.replace('--', '')
# split words by white space
tokens = raw_text.split()
# remove punctuation from each word
re_punc = re.compile('[%s]' % re.escape(string.punctuation))
tokens = [re_punc.sub('', w) for w in tokens]
# remove any non-alphabetic tokens
tokens = [word for word in tokens if word.isalpha()]
print(tokens[:19])

['to', 'sherlock', 'holmes', 'she', 'is', 'always', 'the', 'woman', 'i', 'have', 'seldom', 'heard', 'him', 'mention', 'her', 'under', 'any', 'other', 'name']


In [4]:
len(tokens)

104202

In [5]:
seq_length = 8
words = sorted(list(set(tokens)))
token_to_int = dict((c, i) for i, c in enumerate(words))
dataX = []
dataY = []
n_tokens = len(tokens)
for i in range(0, n_tokens - seq_length, 1):
    seq_in = tokens[i:i + seq_length]
    seq_out = tokens[i + seq_length]
    dataX.append([token_to_int[token] for token in seq_in])
    dataY.append(token_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  104194


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(len(words))

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

print(X.shape[1], X.shape[2])

8 1


In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
104194/104194 [==============================] - 161s 2ms/step - loss: 6.8495

Epoch 00001: loss improved from inf to 6.84947, saving model to weights-improvement-01-6.8495.hdf5
Epoch 2/20
104194/104194 [==============================] - 164s 2ms/step - loss: 6.7335

Epoch 00002: loss improved from 6.84947 to 6.73350, saving model to weights-improvement-02-6.7335.hdf5
Epoch 3/20
104194/104194 [==============================] - 165s 2ms/step - loss: 6.7257

Epoch 00003: loss improved from 6.73350 to 6.72568, saving model to weights-improvement-03-6.7257.hdf5
Epoch 4/20
104194/104194 [==============================] - 166s 2ms/step - loss: 6.6822

Epoch 00004: loss improved from 6.72568 to 6.68217, saving model to weights-improvement-04-6.6822.hdf5
Epoch 5/20
104194/104194 [==============================] - 165s 2ms/step - loss: 6.6070

Epoch 00005: loss improved from 6.68217 to 6.607

In [10]:
# load the network weights
filename = "weights-improvement-20-6.1208.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
import sys

int_to_token = dict((i, c) for i, c in enumerate(words))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" been to command and to be obeyed his "
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
Done.


In [12]:
start2 = numpy.random.randint(576, len(dataX)-1)
pattern = dataX[start2]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" the subject in the course of a day "
of the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
Done.


In [13]:
start3 = numpy.random.randint(1291, len(dataX)-1)
pattern = dataX[start3]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" the hospitality of their tents wandering away with "
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
Done.


In [14]:
start4 = numpy.random.randint(291, len(dataX)-1)
pattern = dataX[start4]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" now he will be here in an hour "
of the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
Done.


In [35]:
start5 = numpy.random.randint(2291, len(dataX)-1)
pattern = dataX[start5]
print("Seed:")
print("\"", ' '.join([int_to_token[value] for value in pattern]), "\"")

# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(words))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_token[index]
    seq_in = [int_to_token[value] for value in pattern]
    sys.stdout.write(result + " ")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" way as ever came before me now let "
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the 
Done.
